In [1]:

from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import mean_squared_error

In [2]:
# run xgboost 
def tree_train(x,y):
    parameters = {"max_depth":[3,4,5,6,7,8,9], "min_child_weight":[1,2,3]}
    model = XGBRegressor()
    gs = GridSearchCV(model,param_grid=parameters,cv=10)
    gs.fit(x,y)
    return gs





In [3]:
def read_data(base_folder='../fpl_data/data',year='2016-17'):
    cleaned_file = base_folder+'/'+year+'/cleaned_players.csv'
    raw_file = base_folder+'/'+year+'/players_raw.csv'
    raw_df = pd.read_csv(raw_file,encoding = "ISO-8859-1")
    #select limited columns only
    raw_df = raw_df[['first_name','second_name','element_type']]
    clean_df = pd.read_csv(cleaned_file,encoding = "ISO-8859-1")
    
    full_df = pd.merge(clean_df,raw_df,on=['first_name','second_name'])
    return full_df

In [4]:
df_2016 = read_data(year='2016-17')
df_2017 = read_data(year='2017-18')
df_2018 = read_data(year='2018-19')

In [5]:
list(df_2016)

['first_name',
 'second_name',
 'goals_scored',
 'assists',
 'total_points',
 'minutes',
 'goals_conceded',
 'creativity',
 'influence',
 'threat',
 'bonus',
 'bps',
 'ict_index',
 'clean_sheets',
 'red_cards',
 'yellow_cards',
 'selected_by_percent',
 'element_type']

## Merge two year to get the y

In [6]:
def merge_df_year(df_1,df_2):
    df_2 = df_2[['first_name','second_name','total_points']]
    # add column name as total_points_new. This is the y value.
    df_2['total_points_next'] = df_2['total_points']
    df_2.drop('total_points',axis=1)
    
    df = pd.merge(df_1,df_2, on =['first_name','second_name'])
    return df

In [7]:
df_2016_17_combined = merge_df_year(df_2016,df_2017)
df_2017_18_combined = merge_df_year(df_2017,df_2018)

/home/shibin/softwares/anaconda2/envs/aiproject/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


# Create train and test data. Data sample is less

In [8]:
def clean_data(x,cols):
    unwanted_cols = ['first_name','second_name','total_points_next']
    for c in list(x):
        if c not in cols:
            unwanted_cols.append(c)
    return x.drop(unwanted_cols,axis=1)

In [12]:
cols = list(df_2016_17_combined)

train_y = df_2016_17_combined['total_points_next']
train_x = clean_data(df_2016_17_combined,cols)
test_y = df_2017_18_combined['total_points_next']
test_x = clean_data(df_2017_18_combined,cols)

#print(list(train_x))

In [13]:
print(cols)

['first_name', 'second_name', 'goals_scored', 'assists', 'total_points_x', 'minutes', 'goals_conceded', 'creativity', 'influence', 'threat', 'bonus', 'bps', 'ict_index', 'clean_sheets', 'red_cards', 'yellow_cards', 'selected_by_percent', 'element_type', 'total_points_y', 'total_points_next']


# Run training

In [14]:
clf = tree_train(train_x,train_y)

In [15]:
prediction = clf.predict(test_x)

In [16]:
error = mean_squared_error(prediction,test_y)

In [17]:
print(error)

0.06651829669993425


In [18]:
def write_result(test_x,test_y,predicted,file='next_year.csv'):
    first_name =test_x['first_name']
    second_name = test_x['second_name']
    df = pd.DataFrame()
    df['first_name'] = first_name
    df['second_name'] = second_name
    df['actual'] = test_y
    df['predicted'] = predicted
    df['element_type'] = test_x['element_type']
    df['diff'] = (df['actual']-df['predicted']).abs()
    df.to_csv(file,index=False)
    return df
    

In [19]:
final_combined = write_result(df_2017_18_combined,test_y,prediction)

# Get players

In [39]:
def select_players(df):
    df = df.sort_values(by=['predicted'],ascending=False)
    result = [[] for x in range(4)]
    required = [2,5,5,3]
    for index,row in df.iterrows():
        f_name = row['first_name']
        s_name = row['second_name']
        pred = row['predicted']
        actual = row['actual']
        e_index = int(row['element_type'])-1
        if(len(result[e_index])<required[e_index]):
            item = {}
            item['first_name'] =f_name
            item['second_name'] =  s_name
            item['prediction'] = pred
            item['actual'] = actual
            result[e_index].append(item)
            
    return result
def print_result(result):
    types = ['Goali','Defender','Mid','Attacker']
    for i in range(len(result)):
        print('==================SELECTED {}==================='.format(types[i]))
        for player in result[i]:
            print('{},{}\t\t\t,{}\n'.format(player['first_name'],player['second_name'],player['prediction']))

In [40]:
s_list = select_players(final_combined)

In [41]:
print_result(s_list)

==================SELECTED Goali===================
Ederson,Santana de Moraes			,71.9899673461914

Jordan,Pickford			,65.28709411621094

==================SELECTED Defender===================
Marcos,Alonso			,93.63746643066406

Andrew,Robertson			,83.39066314697266

Virgil,van Dijk			,74.14910125732422

Trent,Alexander-Arnold			,72.98986053466797

Aymeric,Laporte			,67.04373931884766

==================SELECTED Mid===================
Raheem,Sterling			,104.21186065673828

Eden,Hazard			,93.77366638183594

Mohamed,Salah			,89.39395141601562

Ryan,Fraser			,86.031005859375

Leroy,SanÃ©			,80.99491882324219

==================SELECTED Attacker===================
Pierre-Emerick,Aubameyang			,94.02465057373047

Callum,Wilson			,92.25711059570312

Harry,Kane			,85.42436981201172

